# <span style='color:#547DCD'> Get data from Global Factor Data  </span> 

Import libraries.

In [17]:
import pandas as pd
import wrds

Initialize and connect to Wharton Research Data Services server.

In [ ]:
wrds_db = wrds.Connection()

In [19]:
countries = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Country%20Classification.xlsx')
countries_rel = countries[countries['msci_development'] == 'developed']['excntry'].tolist()

In [6]:
chars = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Factor%20Details.xlsx')
chars_rel = chars[chars['abr_jkp'].notna()]['abr_jkp'].tolist()

In [15]:
sql_query= f"""
          SELECT id, eom, excntry, gvkey, permno, size_grp, me, ret_exc_lead1m
                   FROM contrib.global_factor
                   WHERE common=1 and exch_main=1 and primary_sec=1 and obs_main=1 and
                   excntry='DNK'
          """
data = wrds_db.raw_sql(sql_query)

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)